In [1]:
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, GlobalAveragePooling2D, Flatten, Concatenate, Softmax, Reshape, Lambda
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
import cv2
import moviepy.editor as mp
import os


In [2]:
# Audio Encoder
# 1. Prétraitement de l'audio
# max_length : longueur maximale des MFCCs, correspond à 2 secondes d'audio
def preprocess_audio(video, n_mfcc=40, max_length=173):
    """Split the audio from the video, extract the MFCCs and pad them to max_length"""
    # Charger l'audio
    audio = video.audio
    # Si l'audio est trop court, le remplir avec du silence
    if audio.duration < 2:
        audio = audio.set_duration(2)
    # Si l'audio est plus long que 2 secondes, ne garder que les 2 premières secondes
    else:
        audio = audio.subclip(0, 2)
    # Manipulation temporaire pour sauvegarder l'audio en .wav, obligatoire pour pouvoir charger avec librosa et extraire les MFCCs
    audio.write_audiofile("temp_audio.wav")
    y, sr = librosa.load("temp_audio.wav")
    os.remove("temp_audio.wav")
    # Extraire les MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    # Transposer pour avoir la forme (timesteps, features)
    mfccs = mfccs.T
    # Troncature ou padding pour uniformiser la longueur
    if mfccs.shape[0] > max_length:
        mfccs = mfccs[:max_length, :]
    else:
        padding = max_length - mfccs.shape[0]
        mfccs = np.pad(mfccs, ((0, padding), (0, 0)), mode='constant')
    return mfccs
    # La sortie doit être de forme (None, timesteps, features)
    # return mfccs[np.newaxis, ...]

def AudioEncoder(input_shape=(173,40), encoded_dim=128):
    """
    Modèle d'encodage audio avec un LSTM.
    Arguments:
    - input_shape : forme de l'entrée audio (timesteps, features) ou (features).
    - encoded_dim : dimensions du vecteur encodé.
    """
    inputs = Input(shape=input_shape)

    # Adapter les dimensions si l'entrée est 2D (batch_size, features)
    if len(input_shape) == 1:
        reshaped = Reshape((1, input_shape[0]))(inputs)  # (batch_size, 1, features)
    else:
        reshaped = inputs

    # LSTM pour encoder
    x = LSTM(64, return_sequences=True)(reshaped)
    x = LSTM(64)(x)
    encoded = Dense(encoded_dim, activation='relu')(x)

    # Modèle
    return Model(inputs, encoded, name="AudioEncoder")


In [3]:
# Video Encoder
# 1. Prétraitement des frames
def preprocess_frame(video, target_size=(224, 224)):
    """Split the video to only keep the frame after 1sec and return it as an object variable"""
    # If the file is too short, only keep the first frame
    if video.duration < 1:
        frame = video.get_frame(0)
    else:
        # Only keep the frame after 1 second
        frame = video.get_frame(1)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Convert it to the right color space
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Redimensionner l'image
    frame = cv2.resize(frame, target_size)
    # Normalisation pour ResNet
    frame = tf.keras.applications.resnet.preprocess_input(frame)
    return frame
    # La sortie doit être de forme (None, height, width, channels)
    # Il faut donc ajouter une dimension pour le batch (None)
    # return np.array(frames)[np.newaxis, ...]

# 2. Création du modèle ResNet pour l'encodage
def VideoEncoder(input_shape=(224, 224, 3), encoded_dim=128):
    """Load a ResNet50 model with a GlobalAveragePooling layer and a Dense layer for encoding a frame"""

    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False  # Geler les poids du modèle pré-entraîné
    
    # Ajouter des couches pour obtenir un vecteur encodé
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(encoded_dim, activation='relu')(x)
    
    # Construire le modèle
    model = Model(inputs=base_model.input, outputs=x, name="VideoEncoder")
    return model

In [4]:
# MLP encoder
# 1. Création du MLP Encoder
def MLPEncoder(audio_dim=128, frame_dim=128, output_dim=128):
    # Entrée pour le vecteur audio
    audio_input = Input(shape=(audio_dim,), name="audio_input")
    audio_dense = Dense(64, activation="relu")(audio_input)

    # Entrée pour le vecteur video
    frame_input = Input(shape=(frame_dim,), name="frame_input")
    frames_dense = Dense(256, activation="relu")(frame_input)

    # Fusionner les deux entrées
    merged = Concatenate()([audio_dense, frames_dense])  # Fusionner (64 + 256 = 320)

    # Passages dans des couches MLP pour obtenir le vecteur utilisateur
    x = Dense(256, activation="relu")(merged)
    x = Dense(128, activation="relu")(x)
    user_vector = Dense(output_dim, activation="relu", name="user_vector")(x)

    # Modèle
    model = Model(inputs=[audio_input, frame_input], outputs=user_vector, name="MLPEncoder")
    return model

In [5]:
# Caracterizer (reliant les 3 modèles AudioEncoder, VideoEncoder et MLPEncoder)
def Caracterizer(audio_input_shape=(173,40), audio_encoded_dim=128, frame_input_shape=(224, 224, 3), frame_encoded_dim=128, user_encoded_dim=128):
    # Entrées
    # audio input shape: (None, 173, 40)
    audio_input = Input(shape=audio_input_shape, name="audio_input")
    # frames input shape: (None, 224, 224, 3)
    frame_input = Input(shape=frame_input_shape, name="frame_input")

    # définir les encodeurs
    audio_encoder = AudioEncoder(input_shape=audio_input_shape, encoded_dim=audio_encoded_dim)
    video_encoder = VideoEncoder(input_shape=frame_input_shape, encoded_dim=frame_encoded_dim)
    mlp_encoder = MLPEncoder(audio_dim=audio_encoded_dim, frame_dim=frame_encoded_dim, output_dim=user_encoded_dim)

    # audio_vector = audio_encoder.predict(mfccs)
    # frame_vector = video_encoder.predict(frame)
    # user_vector = mlp_encoder.predict([audio_vector, frame_vector])

    # Lier les encodeurs ensemble
    audio_vector = audio_encoder(audio_input)
    # print('audio_vector shape:', audio_vector.shape)
    # # frames_input shape: (None, 224, 224, 3)
    frame_vector = video_encoder(frame_input)
    # print('frame_vector shape:', frame_vector.shape)
    # # Utilisez une couche Lambda pour empiler les tensors le long de la dimension des frames
    # frames_encoded = Lambda(lambda x: tf.stack(x, axis=1), name="frames_concatenation")(frames_encoded_list)
    # # ajouter une dimension batch au début afin d'avoir la forme (None, 60, 128)
    # frames_encoded = Reshape((num_frames, frame_dim))(frames_encoded)
    
    # MLP Encoder input shape: None, 60, 128
    user_vector = mlp_encoder([audio_vector, frame_vector])

    # Modèle final
    model = Model(inputs=[audio_input, frame_input], outputs=user_vector, name="Caracterizer")
    return model

In [6]:
# MLP Decoder
def MLPDecoder(input_dim=128, num_users=10):
    """
    Crée un modèle MLP Decoder prenant en entrée un vecteur utilisateur (user_vector)
    et renvoyant la probabilité d'appartenance à un utilisateur via une couche softmax.
    
    Arguments :
    - input_dim : Dimension du vecteur utilisateur en entrée.
    - num_users : Nombre maximal d'utilisateurs (#usersmax).
    
    Retour :
    - Modèle Keras du MLP Decoder.
    """
    # Entrée : vecteur utilisateur
    user_vector_input = Input(shape=(input_dim,), name="user_vector_input")
    
    # Couches Dense avec activations ReLU
    x = Dense(128, activation="relu")(user_vector_input)
    x = Dense(64, activation="relu")(x)
    x = Dense(32, activation="relu")(x)
    x = Dense(num_users, activation="relu")(x)  # Dernière couche dense
    
    # Couche Softmax pour obtenir les probabilités
    output = Softmax(name="user_probabilities")(x)
    
    # Modèle
    model = Model(inputs=user_vector_input, outputs=output, name="Decoder")
    return model


In [7]:
# Création du modèle complet
def CompleteModel(audio_input_shape=(173,40), audio_encoded_dim=128, frame_input_shape=(224, 224, 3), frame_encoded_dim=128, user_encoded_dim=128, num_users=10):
    # Créer les modèles
    caracterizer = Caracterizer(audio_input_shape=audio_input_shape, audio_encoded_dim=audio_encoded_dim, frame_input_shape=frame_input_shape, frame_encoded_dim=frame_encoded_dim, user_encoded_dim=user_encoded_dim)
    mlp_decoder = MLPDecoder(input_dim=user_encoded_dim, num_users=num_users)
    
    # Entrées
    audio_input = Input(shape=audio_input_shape, name="audio_input")
    frame_input = Input(shape=frame_input_shape, name="frame_input")
    
    # Caractérisation
    user_vector = caracterizer([audio_input, frame_input])
    
    # Décodage
    user_probabilities = mlp_decoder(user_vector)
    
    # Modèle complet
    model = Model(inputs=[audio_input, frame_input], outputs=user_probabilities)
    return model

In [ ]:
# Exemple d'utilisation pour les modèles séparés (l'audio + la vidéo + le MLP Encoder + le MLP Decoder)
# Dataloader
video_path = 'data/Yann_Zurbrugg/WhatsApp Video 2024-12-02 at 11.41.31.mp4'
# Charger la vidéo
video = mp.VideoFileClip(video_path)
    
# Prétraiter le fichier audio
mfccs = preprocess_audio(video)
mfccs = np.expand_dims(mfccs, axis=0)  # Ajouter une dimension batch
# Prétraiter la vidéo
frame = preprocess_frame(video)
frame = np.expand_dims(frame, axis=0)  # Ajouter une dimension batch

# Créer le modèle
audio_encoder = AudioEncoder(input_shape=(173,40), encoded_dim=128)
video_encoder = VideoEncoder(input_shape=(224, 224, 3), encoded_dim=128)
mlp_encoder = MLPEncoder(audio_dim=128, frame_dim=128, output_dim=128)
num_users = 5  # Nombre maximal d'utilisateurs
decoder = MLPDecoder(input_dim=128, num_users=num_users)

# Résumer les modèles
audio_encoder.summary()
video_encoder.summary()
mlp_encoder.summary()
decoder.summary()

# Passer les données dans le modèle
audio_vector = audio_encoder.predict(mfccs)
frame_vector = video_encoder.predict(frame)
user_vector = mlp_encoder.predict([audio_vector, frame_vector])
user_probabilities = decoder.predict(user_vector)

print("Forme du vecteur audio encodé:", audio_vector.shape)  # (1, 128)
print("Forme du vecteur vidéo encodé :", frame_vector.shape)  # (60, 128)
print("Forme du vecteur utilisateur :", user_vector.shape)  # (1, 128)
# Prédire les probabilités pour chaque utilisateur
print("Probabilités utilisateur :", user_probabilities)

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


Model: "AudioEncoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 173, 40)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 173, 64)        │        26,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 68,224 (266.50 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ user_vector_input (InputLayer)  │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           165 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ user_probabilities (Softmax)    │ (None, 5)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,013 (105.52 KB)

 Trainable params: 27,013 (105.52 KB)

 Non-trainable params: 0 (0.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 684ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Forme du vecteur audio encodé: (1, 128)
Forme du vecteur vidéo encodé : (1, 128)
Forme du vecteur utilisateur : (1, 128)
Probabilités utilisateur : [[0.19971964 0.2000131  0.1965139  0.20272242 0.20103101]]


In [ ]:
# Exemple d'utilisation pour le Characterizer (l'audio + la vidéo + le MLP Encoder)
# Dataloader
video_path = 'data/Yann_Zurbrugg/WhatsApp Video 2024-12-02 at 11.41.31.mp4'
# Charger la vidéo
video = mp.VideoFileClip(video_path)

# Prétraiter l'audio
mfccs = preprocess_audio(video)
mfccs = np.expand_dims(mfccs, axis=0)  # Ajouter une dimension batch
# Prétraiter la vidéo
frame = preprocess_frame(video)
frame = np.expand_dims(frame, axis=0)  # Ajouter une dimension batch

print("Audio input shape:", mfccs.shape)
print("Frame input shape:", frame.shape)

# Création du modèle
caracterizer = Caracterizer(audio_input_shape=(173,40), audio_encoded_dim=128, frame_input_shape=(224, 224, 3), frame_encoded_dim=128, user_encoded_dim=128)

# Résumé du modèle
caracterizer.summary()

# Prédiction
user_vector = caracterizer.predict([mfccs, frame])
print("User vector shape:", user_vector.shape)

MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


Audio input shape: (1, 173, 40)
Frame input shape: (1, 224, 224, 3)


Model: "Caracterizer"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ audio_input         │ (None, 173, 40)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ AudioEncoder        │ (None, 128)       │     68,224 │ audio_input[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VideoEncoder        │ (None, 128)       │ 23,849,984 │ frame_input[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ MLPEncoder          │ (None, 128)       │    172,864 │ AudioEncoder[0][… │
│ (Functional)        │                   │            │ VideoEncoder[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,091,072 (91.90 MB)

 Trainable params: 503,360 (1.92 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
User vector shape: (1, 128)


In [ ]:
# Exemple d'utilisation pour le modèle complet
# Dataloader
video_path = 'data/Yann_Zurbrugg/WhatsApp Video 2024-12-02 at 11.41.31.mp4'
# Charger la vidéo
video = mp.VideoFileClip(video_path)

# Prétraiter l'audio
mfccs = preprocess_audio(video)
mfccs = np.expand_dims(mfccs, axis=0)  # Ajouter une dimension batch
# Prétraiter la vidéo
frame = preprocess_frame(video)
frame = np.expand_dims(frame, axis=0)  # Ajouter une dimension batch

print("Audio input shape:", mfccs.shape)
print("Frame input shape:", frame.shape)

# Création du modèle
model = CompleteModel(audio_input_shape=(173,40), audio_encoded_dim=128, frame_input_shape=(224, 224, 3), frame_encoded_dim=128, user_encoded_dim=128, num_users=10)

# Résumé du modèle
model.summary()

# Prédiction
user_probabilities = model.predict([mfccs, frame])
print("User probabilities shape:", user_probabilities.shape)
print("User probabilities:", user_probabilities)


MoviePy - Writing audio in temp_audio.wav


MoviePy - Done.


Audio input shape: (1, 173, 40)
Frame input shape: (1, 224, 224, 3)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ audio_input         │ (None, 173, 40)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Caracterizer        │ (None, 128)       │ 24,091,072 │ audio_input[0][0… │
│ (Functional)        │                   │            │ frame_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Decoder             │ (None, 10)        │     27,178 │ Caracterizer[0][… │
│ (Functional)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 24,118,250 (92.00 MB)

 Trainable params: 530,538 (2.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
User probabilities shape: (1, 10)
User probabilities: [[0.11130853 0.09578076 0.11076984 0.09558854 0.09558854 0.09558854
  0.10860961 0.09558854 0.09558854 0.09558854]]
